# HuggingTweets - Train a model to generate tweets

Choose your favorite Twitter account and train a language model to write new tweets based on their unique voice in just 5 minutes.

Here is an example where I fine-tuned a neural network to predict Elon Musk's next breakthrough 😉

![huggingtweets illustration](https://raw.githubusercontent.com/borisdayma/huggingtweets/master/img/example.png)

## To start the demo, click on menu at top, "Runtime" → "Run all"

In [ ]:
#@title ⠀ {display-mode: "form"}

def print_html(x):
    "Better printing"
    x = x.replace('\n', '<br>')
    display(HTML(x))
        
# Check we use GPU
import torch
from IPython.display import display, HTML, Javascript, clear_output
if not torch.cuda.is_available():
    print_html('Error: GPU was not found\n1/ click on the "Runtime" menu and "Change runtime type"\n'\
          '2/ set "Hardware accelerator" to "GPU" and click "save"\n3/ click on the "Runtime" menu, then "Run all" (below error should disappear)')
    raise ValueError('No GPU available')
else:
    # Install dependencies
    !pip install wandb transformers torch -qq

    import ipywidgets as widgets
    from IPython import get_ipython
    import json
    import urllib3
    import pathlib
    import shutil
    import requests
    import wandb
    wandb.login(anonymous='allow')  # ensure we log with huggingface
    from transformers import (
        AutoTokenizer, AutoModelForCausalLM,
        TextDataset, DataCollatorForLanguageModeling,
        Trainer, TrainingArguments)
    from transformers.hf_api import HfApi
    
    try:
        import google.colab
        IN_COLAB = True
    except:
        IN_COLAB = False
    
    log_debug = widgets.Output()

    
    # Have global access to model, tokenizer & dataset
    trainer, tokenizer = None, None
    artifact_dataset = None
    card_val = {}

    # Associate run to a project
    with log_debug:
        %env WANDB_PROJECT=huggingtweets
        %env WANDB_WATCH=false
        %env WANDB_ENTITY=borisd13
        %env WANDB_ANONYMOUS=allow
        %env WANDB_NOTEBOOK_NAME=huggingtweets-demo
        %env WANDB_NOTES=Github repo: https://github.com/borisdayma/huggingtweets
        HW_VERSION = 0.3

    def fix_text(text):
        text = text.replace('&amp;', '&')
        text = text.replace('&lt;', '<')
        text = text.replace('&gt;', '>')
        return text

    def html_table(data, title=None):
        'Create a html table'
        width_twitter = '75px'
        def html_cell(i, twitter_button=False):
            return f'<td style="width:{width_twitter}">{i}</td>' if twitter_button else f'<td>{i}</td>'
        def html_row(row):
            return f'<tr>{"".join(html_cell(r, not i if len(row)>1 else False) for i,r in enumerate(row))}</tr>'    
        body = f'<table style="width:100%">{"".join(html_row(r) for r in data)}</table>'
        title_html = f'<h3>{title}</h3>' if title else ''
        html = '''
        <html>
            <head>
                <style>
                    table {border-collapse: collapse !important;}
                    td {text-align:left !important; border: solid #E3F2FD !important; border-width: 1px 0 !important; padding: 6px !important;}
                    tr:nth-child(even) {background-color: #E3F2FD !important;}
                    tr:nth-child(odd) {background-color: #FFFFFF !important;}
                </style>
            </head>
            <body>''' + title_html + body + '</body></html>'
        return(html)

    def clean_tweet(tweet):
        bad_start = ['http', 'pic']
        text = ' '.join(t for t in tweet.split() if not(any(bs in t for bs in bad_start)))
        return text
        
    def boring_tweet(tweet):
        "Check if this is a boring tweet"
        boring_stuff = ['http', '@', '#', 'thank', 'thanks', 'I', 'you']
        if len(tweet.split()) < 3:
            return True
        if all(any(bs in t.lower() for bs in boring_stuff) for t in tweet.split()):
            return True
        return False

    def create_model_card(card_val, output_dir):
        model_card_url = 'https://github.com/borisdayma/huggingtweets/raw/master/model_card/README.md'
        model_card = requests.get(model_card_url).content.decode('utf-8')
        for k, v in card_val.items():
            model_card = model_card.replace(k, v)
        with open(f'{output_dir}/README.md', 'w') as f:
            f.write(model_card)

    def dl_tweets():
        with log_debug:
            wandb.join()  # ensure any previous run is closed
        handle_widget.disabled = True
        run_dl_tweets.disabled = True
        run_dl_tweets.button_style = 'primary'

        get_ipython().kernel.do_one_iteration() # widget slow to update on phones
        handle = handle_widget.value.strip()
        handle = handle[1:] if handle[0] == '@' else handle
        handle = handle.lower().strip()
        log_dl_tweets.clear_output(wait=True)

        success_try = False

        with log_dl_tweets:
            try:
                print_html(f'\nDownloading {handle_widget.value.strip()} tweets... This should take no more than a minute!')
                http = urllib3.PoolManager(retries=urllib3.Retry(3))
                res = http.request("GET", f"http://us-central1-huggingtweets.cloudfunctions.net/get_tweets?handle={handle}&force=1")
                res = json.loads(res.data.decode('utf-8'))
                
                # save user info
                card_val['USER_HANDLE'] = handle
                card_val['USER_NAME'] = res['user_name']
                card_val['USER_PROFILE'] = res['user_profile'].replace('_normal.', '_400x400.')

                all_tweets = res['tweets']
                curated_tweets = [fix_text(tweet) for tweet in all_tweets]
                log_dl_tweets.clear_output(wait=True)
                print_html(f"\n{res['n_tweets']} tweets from {handle_widget.value.strip()} downloaded!\n\n")
                    
                # create dataset
                clean_tweets = [clean_tweet(tweet) for tweet in curated_tweets]
                cool_tweets = [tweet for tweet in clean_tweets if not boring_tweet(tweet)]
                total_text = '<|endoftext|> ' + ' <|endoftext|> '.join(cool_tweets) + ' <|endoftext|>'

                # display a few tweets
                display(HTML(html_table([[t] for t in curated_tweets[:8]])))
                get_ipython().kernel.do_one_iteration() # time for auto-scroll
                
                if len(total_text) < 6000:
                    # need about 4000 chars for one data sample (but depends on spaces, etc)
                    raise ValueError(f"Error: this user does not have enough tweets to train a Neural Network\n{res['n_tweets']} tweets downloaded, including {res['n_RT']} RT's and {len(all_tweets) - len(cool_tweets)} boring tweets... only {len(cool_tweets)} tweets kept!")
                if len(total_text) < 40000:
                    print_html('\n<b>Warning: this user does not have many tweets which may impact the results of the Neural Network</b>')
                    
                print_html('\nCreating dataset...')
                get_ipython().kernel.do_one_iteration() # time for auto-scroll
                    
                # log dataset
                with log_debug:
                    with wandb.init(name=f'@{handle}-dl_data', job_type='dl_data', config={'version':HW_VERSION}, reinit=True) as run:
                        # log raw tweets as input
                        artifact_input = wandb.Artifact(
                            f'tweets-{handle}',
                            type='raw-dataset',
                            description=f'Raw tweets from @{handle} downloaded with Tweepy',
                            metadata={'handle':handle,
                                      'tweets downloaded': res['n_tweets'],
                                      'retweets': res['n_RT'],
                                      'tweets kept': len(all_tweets),
                                      'huggingtweets version': HW_VERSION})
                        with artifact_input.new_file('tweets.txt') as f:
                            json.dump(all_tweets, f, indent=0)
                        run.log_artifact(artifact_input)

                    with wandb.init(name=f'@{handle}-preprocess', job_type='preprocess', config={'version':HW_VERSION}, reinit=True) as run:
                        run.use_artifact(artifact_input)
                        # log dataset as output
                        global artifact_dataset
                        artifact_dataset = wandb.Artifact(
                            f'dataset-{handle}',
                            type='train-dataset',
                            description=f'Dataset created from tweets of @{handle}',
                            metadata={'handle':handle,
                                      'tweets downloaded': res['n_tweets'],
                                      'retweets': res['n_RT'],
                                      'short tweets': len(all_tweets) - len(cool_tweets),
                                      'tweets kept': len(cool_tweets),
                                      'huggingtweets version': HW_VERSION})
                        with open(f'data_{handle}_train.txt', 'w') as f:
                            f.write(total_text)
                        artifact_dataset.add_file(f'data_{handle}_train.txt')
                        run.log_artifact(artifact_dataset)
                        
                        # keep track of url
                        wandb_url = wandb.run.get_url()
                        # remove api token
                        i = wandb_url.find('?apiKey=')
                        wandb_url = wandb_url[:i if i != -1 else None]
                        card_val['WANDB_PREPROCESS'] = wandb_url

                    # Save data info
                    card_val['TWEETS_DL'] = str(res['n_tweets'])
                    card_val['RETWEETS'] = str(res['n_RT'])
                    card_val['SHORT_TWEETS'] = str(len(all_tweets) - len(cool_tweets))
                    card_val['TWEETS_KEPT'] = str(len(cool_tweets))
                
                success_try = True

            except Exception as e:
                print('\nAn error occured...\n')
                print(e)
                run_dl_tweets.button_style = 'danger'
        
        if success_try:
            run_dl_tweets.button_style = 'success'
            log_finetune.clear_output(wait=True)
            with log_finetune:
                print_html('\nFine-tune your model by clicking on "Train Neural Network"')
            run_finetune.disabled = False
            print_html(f"\n🎉 Dataset created: {res['n_tweets']} tweets downloaded, including {res['n_RT']} RT's and {len(all_tweets) - len(cool_tweets)} short tweets... keeping {len(cool_tweets)} tweets")

        handle_widget.disabled = False
        run_dl_tweets.disabled = False
                
    handle_widget = widgets.Text(value='@elonmusk',
                                placeholder='Enter twitter handle')

    run_dl_tweets = widgets.Button(
        description='Download tweets',
        button_style='primary')
    def on_run_dl_tweets_clicked(b):
        dl_tweets()
    run_dl_tweets.on_click(on_run_dl_tweets_clicked)

    log_restart = widgets.Output()
    log_dl_tweets = widgets.Output()
    
    def finetune():
        if run_finetune.button_style == 'success':
            # user double clicked before start of function
            return

        handle_widget.disabled = True
        run_dl_tweets.disabled = True
        run_finetune.disabled = True
        run_finetune.button_style = 'primary'

        handle = handle_widget.value.strip()
        handle = handle[1:] if handle[0] == '@' else handle
        handle = handle.lower()
        log_finetune.clear_output(wait=True)
        clear_output(wait=True)

        success_try = False

        with log_finetune:
            print_html(f'\nTraining Neural Network on {handle_widget.value.strip()} tweets... This could take up to 2-3 minutes!\n')
            progress = widgets.FloatProgress(value=0.1, min=0.0, max=1.0, bar_style = 'info')
            display(progress)

        with log_debug:
            try:                
                # Setting up pre-trained neural network
                with log_finetune:
                    print_html('\nSetting up pre-trained neural network...')
                global trainer, tokenizer
                tokenizer = AutoTokenizer.from_pretrained('gpt2')
                model = AutoModelForCausalLM.from_pretrained('gpt2', cache_dir=pathlib.Path('cache').resolve())
                block_size = tokenizer.max_len
                train_dataset = TextDataset(tokenizer=tokenizer, file_path=f'data_{handle}_train.txt', block_size=block_size, overwrite_cache=True)
                data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
                epochs = 4  # limit before overfitting
                seed = 123
                training_args = TrainingArguments(
                    output_dir=f'output/{handle}',
                    overwrite_output_dir=True,
                    do_train=True,
                    num_train_epochs=epochs,
                    per_device_train_batch_size=1,
                    logging_steps=5,
                    save_steps=0,
                    seed=seed)
                
                # create wandb run (before it's done automatically by Trainer)
                run = wandb.init(name=f'@{handle}-train', job_type='train', config={'huggingtweets version':HW_VERSION, **vars(training_args)}, reinit=True)
                
                # keep track of url
                wandb_url = wandb.run.get_url()
                # remove api token
                i = wandb_url.find('?apiKey=')
                wandb_url = wandb_url[:i if i != -1 else None]
                card_val['WANDB_TRAIN'] = wandb_url

                trainer = Trainer(
                    model=model,
                    args=training_args,
                    data_collator=data_collator,
                    train_dataset=train_dataset,
                    prediction_loss_only=True)
                progress.value = 0.3

                # log dataset and pretrained model
                run.use_artifact(artifact_dataset)
                artifact_gpt2 = wandb.Artifact(
                    f'gpt2',
                    type='pretrained-model',
                    description=f'Pretrained model from OpenAI downloaded from 🤗 Transformers: https://huggingface.co/gpt2',
                    metadata={'huggingtweets version': HW_VERSION})
                artifact_gpt2.add_dir('cache', name='gpt2')
                run.use_artifact(artifact_gpt2)
                progress.value = 0.4
                
                p_start, p_end = 0.4, 0.8
                def progressify(f):
                    "Control progress bar when calling f"
                    def inner(*args, **kwargs):
                        if trainer.epoch is not None:
                            progress.value = p_start + trainer.epoch / epochs * (p_end - p_start)
                        return f(*args, **kwargs)
                    return inner
        
                trainer._training_step = progressify(trainer._training_step)
                
                # Training neural network
                with log_finetune:
                    print_html('Training neural network...\n')
                    display(wandb.jupyter.Run())
                    print_html('\n')
                    display(progress)
                trainer.train()

                # set model config parameters
                trainer.model.config.task_specific_params['text-generation'] = {
                    'do_sample': True,
                    'min_length': 10,
                    'max_length': 160,
                    'temperature': 1.,
                    'top_p': 0.95,
                    'padding_text': '<|endoftext|> '}

                # log fine-tuned model
                artifact_trained = wandb.Artifact(
                    f'model-{handle}',
                    type='finetuned-model',
                    description=f'Model fine-tuned on tweets from @{handle}',
                    metadata={'handle':handle, 'seed':seed, 'huggingtweets version': HW_VERSION})
                trainer.save_model(f'model-{handle}')
                artifact_trained.add_dir(f'model-{handle}', name='model')
                run.log_artifact(artifact_trained)
                progress.value = 0.85
                
                try:
                    # Get token
                    hfapi = HfApi()
                    user, namespace = 'huggingtweets-app', 'huggingtweets'
                    token = hfapi.login(user, namespace)
                    assert hfapi.whoami(token)[0] == user, "Could not log into Hugging Face"

                    # create a folder with model files
                    model_name = handle
                    shutil.rmtree(model_name, ignore_errors=True)
                    trainer.save_model(model_name)
                    tokenizer.save_pretrained(model_name)
                    valid_files = ['config.json',
                                   'pytorch_model.bin',
                                   'special_tokens_map.json',
                                   'tokenizer_config.json',
                                   'vocab.json',
                                   'merges.txt',
                                   'added_tokens.json']
                    for f in pathlib.Path(model_name).glob('*'):
                        if f.name not in valid_files:
                            f.unlink()
                    create_model_card(card_val, model_name)

                    # upload files
                    model_path = pathlib.Path(model_name)
                    assert model_path.is_dir(), f"Expected {model_path} to be a directory"
                    files = [(str(f.resolve()), str(f)) for f in model_path.glob('*')]
                    assert len(files) == 7, "Unexpected number of files in model directory: {len(files)}"
                    for filepath, filename in files:
                        access_url = hfapi.presign_and_upload(token=token, filename=filename, filepath=filepath, organization=namespace)
                        progress.value += 0.02

                except Exception as e:
                    with log_finetune:
                        print_html(f'\n<b>Could not upload the model to Hugging Face</b>\n{e}')

                run_finetune.button_style = 'success'
                run_predictions.disabled = False

                progress.value = 1.0
                progress.bar_style = 'success'
                success_try = True

                with log_finetune:
                    print_html('\n🎉 Neural network trained successfully!')
                log_predictions.clear_output(wait=True)
                with log_predictions:
                    print_html('\nEnter the start of a sentence and click "Run predictions"')
                with log_restart:
                    print_html('\n<b>To change user, refresh the page</b>\n')

            except Exception as e:
                print('\nAn error occured...\n')
                print(e)
                run_finetune.button_style = 'danger'
                run_finetune.disabled = False
                            
        if not success_try:
            display(log_debug)
            progress.bar_style = 'danger'
        
    run_finetune = widgets.Button(
        description='Train Neural Network',
        button_style='primary',
        disabled=True)
    def on_run_finetune_clicked(b):
        finetune()
    run_finetune.on_click(on_run_finetune_clicked)

    log_finetune = widgets.Output()
    with log_finetune:
        print_html('\nWaiting for Step 1 to complete...')

    def clean_prediction(text):
        token = '<|endoftext|>'
        text = text.replace(token, '')
        text = text.strip()
        if text[-1] == '"' and text.count('"') % 2: text = text[:-1]
        return text.strip()

    predictions = []
    
    def shorten_text(text, max_char):
        while len(text) > max_char:
            text = ' '.join(text.split()[:-1]) + '…'
        return text
        
    def predict():
        run_predictions.disabled = True
        start_widget.disabled = True
        run_predictions.button_style = 'primary'
        handle = handle_widget.value.strip()
        handle = handle[1:] if handle[0] == '@' else handle
        handle_uncased = handle.strip()
        handle = handle.lower().strip()
        model_url = f'https://huggingface.co/huggingtweets/{handle}'
        log_predictions.clear_output(wait=True)

        # tweet buttons don't appear well in colab if within log_predictions widget
        # we reset the entire cell
        clear_output(wait=True)
        display(widgets.VBox([start_widget, run_predictions, log_predictions]))

        def tweet_html(tweet_text):
            tweet_text = shorten_text(tweet_text, 238)
            tweet_text = tweet_text.replace('"', '&quot;')

            return '<a href="https://twitter.com/share?ref_src=twsrc%5Etfw" class="twitter-share-button" data-size="large" '\
                    f'data-text="{tweet_text}" '\
                    f'data-url="{model_url}" data-related="borisdayma,weights_biases,huggingface"'\
                    'data-show-count="false">Tweet</a><script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>'

        success_try = False

        # get start sentence
        get_ipython().kernel.do_one_iteration() # widget slow to update on phones
        start = start_widget.value.strip()
                
        with log_predictions:
            print_html(f'\nPerforming predictions of @{handle} starting with "{start}"...\nThis should take no more than 10 seconds!')
        
        with log_debug:
            try:
                # start a wandb run (should never happen)
                if wandb.run is None:
                    wandb.init()

                wandb_url = wandb.run.get_url()
                # remove api token
                i = wandb_url.find('?apiKey=')
                wandb_url = wandb_url[:i if i != -1 else None]
                
                # prepare input
                start_with_bos = '<|endoftext|> ' + start
                encoded_prompt = tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
                encoded_prompt = encoded_prompt.to(trainer.model.device)

                # prediction
                output_sequences = trainer.model.generate(
                    input_ids=encoded_prompt,
                    max_length=160,
                    min_length=10,
                    temperature=1.,
                    top_p=0.95,
                    do_sample=True,
                    num_return_sequences=8
                    )
                stop_token = '<|endoftext|>'
                generated_sequences = []

                # decode prediction
                for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
                    generated_sequence = generated_sequence.tolist()
                    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
                    limit = text.find(stop_token, 1)
                    text = text[: limit if limit != -1 else None]
                    # we also remove new lines which happen due to pre-trained gpt-2 and twitter scraping
                    limit = text.find('\n')
                    text = text[: limit if limit != -1 else None]
                    generated_sequence = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
                    generated_sequences.append(clean_prediction(generated_sequence))
                
                for i, g in enumerate(generated_sequences):
                    predictions.append([start, ' '.join([start, g])])
                
                # log predictions
                wandb.log({'examples': wandb.Table(data=predictions, columns=['Input', 'Prediction'])})

                # make html table
                tweet_data = [[tweet_html(f'Trained a neural network with huggingtweets on @{handle_uncased}: {start} → {g}'), ' '.join([start, g])] for g in generated_sequences]
                tweet_table = HTML(html_table(tweet_data))

                # make model share table
                tweet_share = f'I created an AI bot of @{handle_uncased} with huggingtweets!\nPlay with my model or create your own!\n\nMade by @borisdayma using @huggingface and @weights_biases'
                link_model = f'<a href="{model_url}" rel="noopener" target="_blank">{model_url}</a>'
                share_data = [[tweet_html(tweet_share),
                               f'🎉 Share @{handle_uncased} model: {link_model} <i>(may take 30 seconds to become active)</i>']]
                share_table = HTML(html_table(share_data))

                run_predictions.button_style = 'success'
                success_try = True
                
            except Exception as e:
                print('\nAn error occured...\n')
                print(e)
                run_predictions.button_style = 'danger'

        if success_try:
            with log_predictions:
                log_predictions.clear_output(wait=True)
                
                # somehow display works one way with Jupyter and one way with colab
                if not IN_COLAB:
                    print_html('\n')
                    display(share_table)
                    print_html('\n<b>Click to share your model or favorite tweets or try new predictions!</b>\n\n')
                    display(tweet_table)
                    print_html('\n<b>Click to share your model or favorite tweets or try new predictions!</b>\n\n')
            if IN_COLAB:
                print_html('\n')
                display(share_table)
                print_html('\n<b>Click to share your model or favorite tweets or try new predictions!</b>\n\n')
                display(tweet_table)
                print_html('\n<b>Click to share your model or favorite tweets or try new predictions!</b>\n\n')
        else:
            display(log_debug)
        
        run_predictions.disabled = False
        start_widget.disabled = False
                
    start_widget = widgets.Text(value='My dream is',
                                placeholder='Start a sentence')

    run_predictions = widgets.Button(
        description='Run predictions',
        button_style='primary',
        disabled=True)
    def on_run_predictions_clicked(b):
        predict()
    run_predictions.on_click(on_run_predictions_clicked)

    log_predictions = widgets.Output()
    with log_predictions:
        print_html('\nWaiting for Step 2 to complete...')

    clear_output(wait=True)
    print_html("🎉 Environment set-up correctly! You're ready to move to Step 1!")

## Step 1 - Enter a Twitter handle

Enter a Twitter handle and click Download tweets. This gives the model a dataset of examples to train on.

In [ ]:
#@title ⠀ {display-mode: "form"}
display(widgets.VBox([handle_widget, run_dl_tweets, log_restart, log_dl_tweets]))

## Step 2 - Train your Neural Network

Fine-tune a language model on your unique set of tweets to generate predictions. The model is downloaded from [HuggingFace Transformers](https://huggingface.co/), an awesome open source library for Natural Language Processing.

In [ ]:
#@title ⠀ {display-mode: "form"}
display(widgets.VBox([run_finetune, log_finetune]))

## Step 3: Generate tweets

Type the beginning of a tweet, press Run predictions, and the model will try to come up with a realistic ending to your tweet.

In [ ]:
#@title ⠀ {display-mode: "form"}
if IN_COLAB:
    display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 2000})'''))
display(widgets.VBox([start_widget, run_predictions, log_predictions]))

Huggingtweets is still in its infancy and will get better over time!

In the future, it will train continuously to become a Twitter expert!

## About

*Built by Boris Dayma*

[![Follow](https://img.shields.io/twitter/follow/borisdayma?style=social)](https://twitter.com/borisdayma)

My main goals with this project are:
* to experiment with how to train, deploy and maintain neural networks in production ;
* to make AI accessible to everyone ;
* to have fun!

For more details, visit the project repository.

[![GitHub stars](https://img.shields.io/github/stars/borisdayma/huggingtweets?style=social)](https://github.com/borisdayma/huggingtweets)

**Disclaimer: this project is not to be used to publish any false generated information but to perform research on Natural Language Generation.**

## Resources

* [Explore the W&B report](https://bit.ly/2TGXMZf) to understand how the model works
* [HuggingFace and W&B integration documentation](https://docs.wandb.com/library/integrations/huggingface)
* [A step by step guide](https://app.wandb.ai/jxmorris12/huggingface-demo/reports/A-Step-by-Step-Guide-to-Tracking-Hugging-Face-Model-Performance--VmlldzoxMDE2MTU): track your Hugging Face model performance

## Got questions about W&B?

If you have any questions about using W&B to track your model performance and predictions, please reach out to the [slack community](http://bit.ly/wandb-forum).